**1. Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.**

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Set path and import data

path = r'/Users/peimei/09-2023 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_prods_merged_updated.pkl'))
ords_prods_merge.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/peimei/09-2023 Instacart Basket Analysis/02 Data/Prepared Data/orders_prods_merged_updated.pkl'

**2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.**

In [ ]:
# Group data by "product_name"

ords_prods_merge.groupby('product_name')

In [ ]:
# Compare the mean values of "order_number" across "department_id"

ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

**3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.**

All mean values in the complete dataframe are lower than those in the subset, except department_id 17.

**4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.**

In [ ]:
# Create a new column "max_order"

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Check the output

ords_prods_merge.head()

In [ ]:
# Use loc() to assign labels

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [ ]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Create a frequency table for column "loyalty_flag"

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

**5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.**

In [ ]:
# Perform multiple aggregations

ords_prods_merge.groupby('loyalty_flag').agg({'prices':['max','mean','min']})

**Observation: Max value (99999) is unexpectedly high! Further investigation on this is necessarry.**

In [ ]:
# Look at the distirbution of prices

ords_prods_merge['prices'].drop_duplicates().sort_values(ascending = False)

**Observations:** The normal price range: 1 - 25. The values "99999" and "14900" are obviously outliers.

In [ ]:
# Frequency table for prices

ords_prods_merge['prices'].value_counts(dropna = False)

**Observations:** 
<br>4429 rows of data have the abnormal price of "14900".
<br>798 rows of data have the abnormal price of "99999".
<br>As the total count of rows with abnormal prices is neglectable, removing them is an option.

In [ ]:
# Create a subset of rows with abnormal prices

df_abnormal_prices = ords_prods_merge.loc[ords_prods_merge['prices'].isin([99999,14900])]
df_abnormal_prices.shape

**Observations:** There are 5127 rows with an abnormal price.

In [ ]:
df_abnormal_prices['product_id'].value_counts(dropna = False)

**Observations:** Two products with an abnormal price are identified.

In [ ]:
# Check if the product_id 21553 has only one price value

df_product_id_21553 = ords_prods_merge.loc[ords_prods_merge['product_id'].isin([21553])]
df_product_id_21553['prices'].value_counts(dropna = False)

**Observations:** The product_id 21553 only has one price value, i.e. the abnormal value of 14900. Thus, imputing the value would take much time. 

In [ ]:
# Check if the product_id 33664 only has one price value

df_product_id_33664 = ords_prods_merge.loc[ords_prods_merge['product_id'].isin([33664])]
df_product_id_33664['prices'].value_counts(dropna = False)

**Observations:** The product_id 33664 only has one price value, i.e. the abnormal value of 99999. Thus, imputing the value would take much time. 

In [ ]:
# Remove rows with abnormal prices and update the dataframe
# Confirm the output

abnormal_prices = [99999,14900]
ords_prods_merge.drop(ords_prods_merge[ords_prods_merge['prices'].isin(abnormal_prices)].index, inplace = True)
ords_prods_merge['prices'].max()

In [ ]:
# Perform multiple aggregations again

ords_prods_merge.groupby('loyalty_flag').agg({'prices':['max','mean','min']})

**My observations:** 
<br> 1. All three types of customers have the same max value and the same min value in the prices of products they purchased. 
<br> 2. The three types of customers spent almost the same amount of money on average, with less than 0.03 (dollars) between the highest spending group (new customers) and the lowest spending group (loyal customers).

**6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”**

In [ ]:
# Calculate the mean value

ords_prods_merge['price_mean'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# Check the outpot

pd.options.display.max_rows = None 
ords_prods_merge.head(100)

In [ ]:
# Create a spending flag

ords_prods_merge.loc[ords_prods_merge['price_mean'] < 10, 'spending_flag'] = 'Low spender'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['price_mean'] >= 10, 'spending_flag'] = 'High spender'

In [ ]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

**7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”**

In [ ]:
# Create a new column "median_days_since_last_order"
# Check the output

ords_prods_merge['median_days_since_last_order'] = ords_prods_merge.groupby(['user_id'])['no_of_days_since_last_order'].transform(np.median)
ords_prods_merge.head()

In [ ]:
# Create order frequency flag

ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [ ]:
ords_prods_merge.loc[(ords_prods_merge['median_days_since_last_order'] <= 20) & (ords_prods_merge['median_days_since_last_order'] > 10), 'order_frequency_flag'] = 'Regular customer'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [ ]:
# Frequeny table for "order_frequency_flag"

ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

In [ ]:
# Check the records with null values

ords_prods_merge.loc[ords_prods_merge['order_frequency_flag'].isna()]

In [ ]:
# Investigate user_id 159838

ords_prods_merge.loc[ords_prods_merge['user_id']== 159838]

**Observations:** 
<br>1. Looking at the complete history of this customer's orders, we can assume that this customer has only made one order and never returned. This can explain why there is no further values in the column "no_of_days_since_last_order" for this customer.
<br> 2. As this is a normal incident, we can accept the output for "order_frequency_flag" as produced.

**8. Ensure your notebook is clean and structured and that your code is well commented.**

Done.

**9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.**

In [ ]:
# Save dataframe as pkl file

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_updated_2.pkl'))